# A Simple Program to pull songs and their attributes from a playlist

[https://developer.spotify.com/dashboard](https://developer.spotify.com/dashboard)

Login and create an app (I used Web API and 'http://localhost:8888/callback' as a throwaway redirect URI)
Record the Client ID and Client Secret.
You can get the playlist ID from the share link on your playlist in Spotify. It's the end of the address:
https://open.spotify.com/playlist/4zE5WtdaS1NMPv3TILL46q

'4zE5WtdaS1NMPv3TILL46q'


In [41]:
# Add the imports for the libraries you'll be using
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# User data to enter for the Spotify API access uses strings, so keep quotes around the values
CLIENTID = 'YOUR_CLIENT_ID'
CLIENTSECRET = 'YOUR_CLIENT_SECRET'
PLAYLISTID = 'YOUR_PLAYLIST_ID'

In [32]:
# Replace 'YOUR_CLIENT_ID' and 'YOUR_CLIENT_SECRET' with your actual client credentials
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENTID,
                                                      client_secret=CLIENTSECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [33]:
def get_playlist_tracks(playlist_id):
    all_tracks = []
    offset = 0
    while True:
        tracks = sp.playlist_tracks(playlist_id, offset=offset)
        all_tracks.extend(tracks['items'])
        offset += len(tracks['items'])
        if not tracks['next']:
            break
    playlist_details = []
    for item in all_tracks:
        track = item['track']
        track_details = {
            'Track Name': track['name'],
            'Artist(s)': ', '.join([artist['name'] for artist in track['artists']]),
            'Album': track['album']['name'],
            'Release Date': track['album']['release_date'],
            'Popularity': track['popularity'],
            'Track ID': track['id']
        }
        playlist_details.append(track_details)
    return playlist_details

# Function to get audio features for a track
def get_audio_features(track_id):
    audio_features = sp.audio_features(track_id)[0]
    return audio_features

# Replace 'PLAYLIST_ID' with the ID of your playlist
playlist_id = PLAYLISTID
playlist_tracks_details = get_playlist_tracks(playlist_id)

# Iterate through each track to get audio features
playlist_audio_features = []
for track_details in playlist_tracks_details:
    track_id = track_details['Track ID']
    audio_features = get_audio_features(track_id)
    playlist_audio_features.append(audio_features)

# Convert the lists of dictionaries into pandas DataFrames
playlist_tracks_df = pd.DataFrame(playlist_tracks_details)
playlist_audio_features_df = pd.DataFrame(playlist_audio_features)

In [34]:
# rename the id column to Track ID to match the playlist_tracks_df feature
playlist_audio_features_df.rename(columns={'id': 'Track ID'}, inplace=True)

In [35]:
# compbine the columns of the two Dataframes by Track ID
# These are unique and each song only has one row in each dataframe
playlist_df = pd.merge(playlist_tracks_df, playlist_audio_features_df, on='Track ID')

In [36]:
# show the df sorted by popularity and show the top 10
playlist_df.sort_values(by='Popularity', ascending=False).head(10)

,Track Name,Artist(s),Album,Release Date,Popularity,Track ID,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
365,Every Breath You Take,The Police,Synchronicity (Remastered 2003),1983-06-17,90,1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,...,0.002940,0.0714,0.740,117.401,audio_features,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,https://api.spotify.com/v1/tracks/1JSTJqkT5qHq...,https://api.spotify.com/v1/audio-analysis/1JST...,253920,4
340,Zombie,The Cranberries,No Need To Argue (The Complete Sessions 1994-1...,1994-10-03,86,7EZC6E7UjZe63f1jRmkWxt,0.309,0.649,7,-8.994,...,0.005910,0.3670,0.312,166.915,audio_features,spotify:track:7EZC6E7UjZe63f1jRmkWxt,https://api.spotify.com/v1/tracks/7EZC6E7UjZe6...,https://api.spotify.com/v1/audio-analysis/7EZC...,306440,4
160,Fast Car,Tracy Chapman,Tracy Chapman,1988-04-05,86,2M9ro2krNb7nr7HSprkEgo,0.711,0.292,4,-15.523,...,0.000000,0.1310,0.194,103.951,audio_features,spotify:track:2M9ro2krNb7nr7HSprkEgo,https://api.spotify.com/v1/tracks/2M9ro2krNb7n...,https://api.spotify.com/v1/audio-analysis/2M9r...,296800,4
469,Tennessee Whiskey,Chris Stapleton,Traveller,2015-05-04,85,3fqwjXwUGN6vbzIwvyFMhx,0.392,0.370,9,-10.888,...,0.009600,0.0821,0.512,48.718,audio_features,spotify:track:3fqwjXwUGN6vbzIwvyFMhx,https://api.spotify.com/v1/tracks/3fqwjXwUGN6v...,https://api.spotify.com/v1/audio-analysis/3fqw...,293293,4
336,No Surprises,Radiohead,OK Computer,1997-05-28,85,10nyNJ6zNy2YVYLrcwLccB,0.255,0.393,5,-10.654,...,0.003610,0.1130,0.118,76.426,audio_features,spotify:track:10nyNJ6zNy2YVYLrcwLccB,https://api.spotify.com/v1/tracks/10nyNJ6zNy2Y...,https://api.spotify.com/v1/audio-analysis/10ny...,229120,4
178,Fade Into You,Mazzy Star,So Tonight That I Might See,1993-10-05,84,1LzNfuep1bnAUR9skqdHCK,0.420,0.330,9,-13.925,...,0.000679,0.0677,0.175,157.143,audio_features,spotify:track:1LzNfuep1bnAUR9skqdHCK,https://api.spotify.com/v1/tracks/1LzNfuep1bnA...,https://api.spotify.com/v1/audio-analysis/1LzN...,295600,3
217,"Rocket Man (I Think It's Going To Be A Long, L...",Elton John,Honky Chateau,1972-05-19,84,3gdewACMIVMEWVbyb8O9sY,0.601,0.532,10,-9.119,...,0.000006,0.0925,0.341,136.571,audio_features,spotify:track:3gdewACMIVMEWVbyb8O9sY,https://api.spotify.com/v1/tracks/3gdewACMIVME...,https://api.spotify.com/v1/audio-analysis/3gde...,281613,4
346,Vienna,Billy Joel,The Stranger (Legacy Edition),1977,83,4U45aEWtQhrm8A5mxPaFZ7,0.532,0.495,10,-6.662,...,0.000000,0.0754,0.308,124.936,audio_features,spotify:track:4U45aEWtQhrm8A5mxPaFZ7,https://api.spotify.com/v1/tracks/4U45aEWtQhrm...,https://api.spotify.com/v1/audio-analysis/4U45...,214240,4
170,(Sittin' On) the Dock of the Bay,Otis Redding,The Dock of the Bay (Mono),1968,83,3zBhihYUHBmGd2bcQIobrF,0.769,0.367,2,-11.226,...,0.000016,0.0810,0.535,103.621,audio_features,spotify:track:3zBhihYUHBmGd2bcQIobrF,https://api.spotify.com/v1/tracks/3zBhihYUHBmG...,https://api.spotify.com/v1/audio-analysis/3zBh...,163756,4
281,Your Song,Elton John,Elton John,1970-04-10,83,38zsOOcu31XbbYj9BIPUF1,0.554,0.315,3,-11.028,...,0.000006,0.1010,0.325,128.157,audio_features,spotify:track:38zsOOcu31XbbYj9BIPUF1,https://api.spotify.com/v1/tracks/38zsOOcu31Xb...,https://api.spotify.com/v1/audio-analysis/38zs...,241787,4


In [37]:
# Check the columns and data types of the DataFrame
playlist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 591
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        592 non-null    object 
 1   Artist(s)         592 non-null    object 
 2   Album             592 non-null    object 
 3   Release Date      592 non-null    object 
 4   Popularity        592 non-null    int64  
 5   Track ID          592 non-null    object 
 6   danceability      592 non-null    float64
 7   energy            592 non-null    float64
 8   key               592 non-null    int64  
 9   loudness          592 non-null    float64
 10  mode              592 non-null    int64  
 11  speechiness       592 non-null    float64
 12  acousticness      592 non-null    float64
 13  instrumentalness  592 non-null    float64
 14  liveness          592 non-null    float64
 15  valence           592 non-null    float64
 16  tempo             592 non-null    float64
 1

In [38]:
# retype the release date to datetime
playlist_df['Release Date'] = pd.to_datetime(playlist_df['Release Date'])

# create a column of just the year of the release date
playlist_df['Release Year'] = playlist_df['Release Date'].dt.year